# Alkali Density Error Analysis

## Data Shape
A processed data file will contain the following 
- a set of magnetic field values (in Gauss)
- a set of rotation values (in radians)
- a set of rotation error values, calculated using mean absolute error
- a set of rotation error values, calculated using standard deviation

In [399]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import density_app.functions.utilities as util


def get_processed_data(filepath):
    data = pd.read_csv(filepath)
    x_data = data["Magnetic Field (Gauss)"].to_numpy()
    y_data = data["Rotation (Radians)"].to_numpy()
    err_MAE = data["Rotation Mean Absolute Error"].to_numpy()  
    err_STD = data["Rotation Standard Deviation"].to_numpy()
    return x_data, y_data, err_MAE, err_STD
#f = "Data/Density/2024-07-24/2024-07-24_cell-314A_temp-110_trial-1_processed.csv"
f = "Data/Density/2024-08-12/2024-08-12_cell-314B_temp-130_trial-1_processed.csv"
x, y, err_1, err_2 = get_processed_data(f)


### Magnetic Field
Magnetic field is calculated from recorded current values using a linear equation based on EPR data for our specific magnetic coil. 
`bfield = float(current) * 2.17`

### Rotation
Each rotation value comes from a set of voltage values (typically 10, but this can be changed) taken from an oscilloscope. The rotation is then the average voltage (which is later transformed into the correct units using calibration values collected at the beginning of the experiment).

### Error Calculations
Since the rotation value is based on an average voltage value, the error can be calculated from the collection of values used to get that average. This is currently done in two ways.

#### Mean Absolute Error
$$
MAE = \frac{\sum^N_i|(\bar{x}-x_i)|}{N}
$$

#### Standard Deviation
$$
\sigma = \sqrt{\frac{1}{N}\sum^N_i(x_i-\bar{x})^2}
$$

In [400]:
# Using curve_fit from scipy optimize to obtain a slope and error on the slope from a dataset

#returns a linear fit for the specified data
#this version uses error values provided for the rotation values
def linear_fit_data(x_vals, y_vals, errors):
    # line for fitting
    def line(x, m, b):
        return x*m+b
    param, param_cov = curve_fit(line, x_vals, y_vals, sigma=errors, absolute_sigma=True)
    return param, param_cov

#returns a linear fit for the specified data
#this does not include error values 
def linear_fit_data_no_errs(x_vals, y_vals):
    # line for fitting
    def line(x, m, b):
        return x*m+b
    param, param_cov = curve_fit(line, x_vals, y_vals)
    return param, param_cov

p1, c1 = linear_fit_data(x, y, err_1)
p2, c2 = linear_fit_data(x, y, err_2)
p3, c3 = linear_fit_data_no_errs(x, y)

df = pd.DataFrame({"Slope":[util.formatter(p1[0],6), util.formatter(p2[0],6),util.formatter(p3[0],6)],
                  "Intercept":[util.formatter(p1[1],6), util.formatter(p2[1],6),util.formatter(p3[1],6)],
                  "SlopeVariance": [c2[0][0], c2[0][0], c3[0][0]],
                  "Rotation Error Type":["mean absolute error", "standard deviation", "none"]}).set_index("Rotation Error Type")

df.head()

,Slope,Intercept,SlopeVariance
Rotation Error Type,,,
mean absolute error,3.930109e-05,6.946785e-06,8.571883e-16
standard deviation,3.935019e-05,7.575566e-06,8.571883e-16
none,3.928533e-05,3.727528e-06,5.829946e-14


## Statistics!

### Sum of Squares Terms
The x and y terms are given by: 
$$
SS_{xx} = \sum^N_i(x_i - \bar{x})^2
$$

$$
SS_{yy} = \sum^N_i(y_i - \bar{y})^2
$$

### Mixed Term for Sum of Squares
The mixed term is given by: 
$$
SS_{xy} = \sum^N_i(x_i - \bar{x})(y_i - \bar{y})
$$

### The Least Squares Fit Slope
The slope $\hat{m}$ of the least squares fit line is given by
$$
\hat{m} = \frac{SS_{xy}}{SS_{xx}}
$$

### The Least Squares Fit Intercept
The intercept, $\hat{b}$ is given by
$$
\hat{b} = \bar{y}-\hat{m}\bar{x}
$$
where $\bar{x}$ and $\bar{y}$ are the average values of x and y respectively. 

### Variance in Y
The variance in y is given by summing the squares of the difference between the measured y, $y_i$, and the y value given by the fit line. 
$$
\sigma_{y}^2 = \frac{1}{N-2} \sum(y_i - (\hat{m}x_i + \hat{b}))^2
$$

### Variance in the Slope
The variance in the slope is then found by dividing the variance in y by the sum of sqares in x. 
$$
\sigma_{slope}^2 = \frac{\sigma_{y}^2}{SS_{xx}} 
$$

### The Code
In the code below, the function `get_fit_values()` takes two arrays representing the x and y data and returns the least squares fit slope, the least squares fit intercept, and the variance in the slope calculated using the methods shown in this section. 



In [401]:
# CODING THE STATISTICS 

# Sum of squares
def sum_sqrs(ary):
    sqrs = 0
    a_bar = np.average(ary)
    for a in ary:
        sqrs = sqrs + (a-a_bar)**2
    return sqrs

#Sum of squares mixed term
def sum_mixed_sqrs(ary1, ary2):
    mix = 0
    a1_bar = np.average(ary1)
    a2_bar = np.average(ary2)
    for i in range(0, len(ary1)):
        mix = mix + (ary1[i]-a1_bar)*(ary2[i]-a2_bar)
    return mix

#variance in y
def variance_y(x, y, m, b):
    var_y = 0
    for i in range(0, len(x)):
        fit_y = m*x[i]+b
        sq_diff = (y[i]-fit_y)**2
        var_y = var_y + sq_diff
    return var_y/(len(x)-2)

#combine all the above functions to get slope, intercept, variance in slope
def get_fit_values(ary1, ary2):
    a1_bar = np.average(ary1)
    a2_bar = np.average(ary2)
    ssxx = sum_sqrs(ary1)
    ssxy = sum_mixed_sqrs(ary1,ary2)
    slope = ssxy/ssxx
    intercept = a2_bar-slope*a1_bar #y_bar-slope*x_bar
    var_slope = variance_y(ary1, ary2, slope, intercept)/ssxx #variance in y / sum of sqares xx
    return slope, intercept, var_slope


## Using the Code to Verify curve_fit
Using previously collected data, calculate the least squares fit slope, intercept and slope variance. And then compare those to the values calculated by scipy's curve_fit function. 
This version compares curve_fit using the default error settings to my calculations for a data set without considering the error on each point.

In [402]:
m, b, v = get_fit_values(x,y)

def percent_dif(est, act):
    p = 100*abs((act-est)/act)
    p = util.formatter(p,2)
    return p


slope_compare = percent_dif(m, p3[0])
intercept_compare = percent_dif(b, p3[1])
variance_compare = percent_dif(v, c3[0][0])

slopes = [util.formatter(m,6), util.formatter(p3[0],6), slope_compare]
intercepts = [util.formatter(b,6), util.formatter(p3[1],6), intercept_compare]
slope_variance = [util.formatter(v,6), util.formatter(c3[0][0],6), variance_compare]


res = pd.DataFrame({"":["My calculation:", "curve_fit's calculation: ", "percent difference:"], 
       "Slope":slopes, 
       "Intercept":intercepts, 
       "Slope Variance":slope_variance}).set_index("")

res.head()


# maybe add some percent differences here to demonstrate that we're well within rounding error or whatever

,Slope,Intercept,Slope Variance
,,,
My calculation:,3.928533e-05,3.727528e-06,5.829946e-14
curve_fit's calculation:,3.928533e-05,3.727528e-06,5.829946e-14
percent difference:,1.10e-09,7.69e-06,4.56e-08


## More Statistics!
Now we want to confim that everything still works how we expect if we include the error on the individual points in our fit line. 

### Least Squares Slope, with errors

Including weighting from the error values, the slope of the least squares fit is given by
$$
\hat{m} = \frac{\sum\frac{x_i}{e_i^2}\sum\frac{y_i}{e_i^2} - \sum\frac{x_iy_i}{e_i^2}\sum\frac{1}{e_i^2}}{(\sum\frac{x_i}{e_i^2})^2-\sum\frac{x_i^2}{e_i^2}\sum\frac{1}{e_i^2}}
$$

where $e_i$ is the error on each individual y value. 

### Least Sqares Intercept, with error

The intercept, including weighting from the errors is given by

$$
\hat{b} = \frac{\sum\frac{x_iy_i}{e_i^2} - \hat{m}\sum\frac{x_i^2}{e_i^2}}{\sum\frac{x_i}{e_i^2}}
$$

### Slope Variance, with error
Encorporating the errors, the slope variance is found by,
$$
\sigma_{slope}^2 = \frac{\sum\frac{1}{e_i^2}}{\sum\frac{x_i^2}{e_i^2}\sum\frac{1}{e_i^2}-(\sum\frac{x_i}{e_i^2})^2}
$$

### The Code
The following code calculates the slope, intercept, and slope variance for a given data set, with error provided for the y values using the equations detailed above. 

In [403]:
def get_fit_values_with_error(ary1, ary2, errors):
    x_er_sum = np.sum(ary1/errors**2)
    y_er_sum = np.sum(ary2/errors**2)
    xy_er_sum = np.sum((ary1*ary2)/errors**2)
    inv_er_sum = np.sum(1/errors**2)
    x_er_sqd_sum = np.sum(ary1**2/errors**2)
    slope = ((x_er_sum*y_er_sum)-(xy_er_sum*inv_er_sum))/(x_er_sum**2-(x_er_sqd_sum*inv_er_sum))
    intercept = (xy_er_sum - slope*x_er_sqd_sum)/x_er_sum
    var_slope = (inv_er_sum)/((x_er_sqd_sum*inv_er_sum)-(x_er_sum)**2)
    return slope, intercept, var_slope


## Verifying curve_fit, but with errors

The code below uses the same data set as before, but now includes the error in the rotation when calculating the fit values. 

In [404]:
m1,b1,v1= get_fit_values_with_error(x,y,err_1)

# Using Mean absolute error on y_i
slope_comp1 = percent_dif(m1, p1[0])
intercept_comp1 = percent_dif(b1, p1[1])
var_comp1 = percent_dif(v1, c1[0][0])
slopes1 = [util.formatter(m1, 6), util.formatter(p1[0],6),slope_comp1]
intercepts1 = [util.formatter(b1, 6), util.formatter(p1[1],6),intercept_comp1]
vars1 = [util.formatter(v1, 6), util.formatter(c1[0][0],6), var_comp1]

df1 = pd.DataFrame({"":["my calculation", "curve_fit's calculation", "percent difference"],
      "Slope":slopes1,
      "Intercept":intercepts1,
      "Slope Variance":vars1}).set_index("")

print("Using MAE")
df1.head()





Using MAE


,Slope,Intercept,Slope Variance
,,,
my calculation,3.930109e-05,6.946787e-06,5.877076e-16
curve_fit's calculation,3.930109e-05,6.946785e-06,5.877074e-16
percent difference,2.47e-07,2.77e-05,4.19e-05


In [405]:
m2,b2,v2 = get_fit_values_with_error(x,y,err_2)

# Using standard deviation on y_i
slope_comp2 = percent_dif(m2, p2[0])
intercept_comp2 = percent_dif(b2, p2[1])
var_comp2 = percent_dif(v2, c2[0][0])
slopes2 = [util.formatter(m2, 6), util.formatter(p2[0],6),slope_comp2]
intercepts2 = [util.formatter(b2, 6), util.formatter(p2[1],6),intercept_comp2]
vars2 = [util.formatter(v2, 6), util.formatter(c2[0][0],6), var_comp2]

df2 = pd.DataFrame({"":["my calculation", "curve_fit's calculation", "percent difference"],
      "Slope":slopes2,
      "Intercept":intercepts2,
      "Slope Variance":vars2}).set_index("")

print("Using Standard Deviation")
df2.head()


Using Standard Deviation


,Slope,Intercept,Slope Variance
,,,
my calculation,3.935019e-05,7.575566e-06,8.571883e-16
curve_fit's calculation,3.935019e-05,7.575566e-06,8.571883e-16
percent difference,6.25e-10,2.47e-07,3.52e-06


### Further Analysis

To make sure that the one file I tested all of this on wasn't a fluke, I want to calculate the curve_fit fit values as well as the fit values from my own coding of the least squares fit for several data sets. 

In [406]:
#just to be extra sure, lets do this analysis on a whole bunch of 
# files and just collect the percent differences by data type
# and present that

## for simplicity I will only compare the DIY and curveift 
# versions with the std error rather than doing all 3 types for each file

files = ["Data/Density/2024-08-12/2024-08-12_cell-314A_temp-80_trial-2_processed.csv", 
         "Data/Density/2024-08-12/2024-08-12_cell-314A_temp-90_trial-3_processed.csv", 
         "Data/Density/2024-08-12/2024-08-12_cell-314A_temp-100_trial-4_processed.csv",
         "Data/Density/2024-08-12/2024-08-12_cell-314A_temp-110_trial-5_processed.csv",
         "Data/Density/2024-08-12/2024-08-12_cell-314A_temp-120_trial-6_processed.csv",
         "Data/Density/2024-08-12/2024-08-12_cell-314A_temp-130_trial-7_processed.csv",
         "Data/Density/2024-08-12/2024-08-12_cell-314B_temp-130_trial-1_processed.csv",
         "Data/Density/2024-08-09/2024-08-09_cell-314B_temp-80_trial-1_processed.csv",
         "Data/Density/2024-08-09/2024-08-09_cell-314B_temp-90_trial-2_processed.csv",
         "Data/Density/2024-08-09/2024-08-09_cell-314B_temp-100_trial-4_processed.csv",
         "Data/Density/2024-08-09/2024-08-09_cell-314B_temp-110_trial-3_processed.csv",
         "Data/Density/2024-08-09/2024-08-09_cell-314B_temp-120_trial-5_processed.csv"
         ]

df = pd.DataFrame({"curve_fit slope":[],
                   "curve_fit intercept":[],
                   "curve_fit slope variance":[],
                   "my slope":[],
                   "my intercept":[],
                   "my slope variance":[],
                   "Percent Difference Slopes":[],
                   "Percent Difference Intercepts":[],
                   "Percent Difference Variances":[]})

for f in files:
    #get the data from each file
    x,y,err_1,err_2 = get_processed_data(f)
    params, pcov = linear_fit_data(x, y, err_2) #curve fit vals
    cf_m = params[0]
    cf_b = params[1]
    cf_var = pcov[0][0]
    m, b, var = get_fit_values_with_error(x,y,err_2) #diy values
    slope_comp = percent_dif(m, cf_m)
    int_comp = percent_dif(b, cf_b)
    var_comp = percent_dif(var, cf_var)
    new_row = {"curve_fit slope":util.formatter(cf_m,3),
                   "curve_fit intercept":util.formatter(cf_b, 3),
                   "curve_fit slope variance":util.formatter(cf_var,3),
                   "my slope":util.formatter(m,3),
                   "my intercept": util.formatter(b,3),
                   "my slope variance":util.formatter(var,3),
                   "Percent Difference Slopes":slope_comp,
                   "Percent Difference Intercepts":int_comp,
                   "Percent Difference Variances":var_comp}
    df.loc[len(df)] = new_row

df


,curve_fit slope,curve_fit intercept,curve_fit slope variance,my slope,my intercept,my slope variance,Percent Difference Slopes,Percent Difference Intercepts,Percent Difference Variances
0,5.734e-06,-3.771e-06,2.167e-15,5.734e-06,-3.771e-06,2.167e-15,2.13e-09,4.92e-09,1.13e-06
1,6.888e-06,-1.511e-06,1.339e-15,6.888e-06,-1.511e-06,1.339e-15,3.33e-07,3.44e-05,1.17e-05
2,1.053e-05,6.683e-06,6.894e-16,1.053e-05,6.683e-06,6.894e-16,3.07e-08,3.17e-06,1.44e-06
3,1.727e-05,-1.715e-05,4.093e-16,1.727e-05,-1.715e-05,4.093e-16,9.50e-09,2.75e-07,2.01e-06
4,2.415e-05,4.376e-05,1.318e-15,2.415e-05,4.376e-05,1.318e-15,7.16e-09,2.19e-07,1.50e-06
5,3.944e-05,-3.548e-06,1.753e-15,3.944e-05,-3.548e-06,1.753e-15,6.62e-09,1.80e-06,9.54e-07
6,3.935e-05,7.576e-06,8.572e-16,3.935e-05,7.576e-06,8.572e-16,6.25e-10,2.47e-07,3.52e-06
7,6.819e-06,1.871e-06,7.016e-16,6.819e-06,1.871e-06,7.016e-16,2.26e-08,3.74e-06,9.51e-07
8,8.130e-06,-7.006e-07,4.227e-16,8.130e-06,-7.006e-07,4.227e-16,2.04e-07,2.80e-04,5.93e-06
9,8.859e-06,1.891e-05,4.383e-16,8.859e-06,1.891e-05,4.383e-16,7.86e-08,9.05e-07,1.44e-06


## Conclusion

Comparing the values calculated directly using the standard statistical techniques, the fit values for the slope differ by ~$1\times10^{-7}$% or less, the intercept values differ by ~$1\times10^{-4}$% or less, and the slope variance differs by ~$1\times10^{-5}$% or less. This level of difference is well below the errors we would see on data we collect, and likely can be attributed to rounding differences between what I have written and curve_fit, rather than differences in technique. And so we can conclude that curve_fit is using standard least squares fit techniques. 